In [1]:
import pandas as pd
import numpy as np
import itertools
import random
import ast
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, wilcoxon, friedmanchisquare, kendalltau, pearsonr
from matplotlib import animation
from mpl_toolkits import mplot3d
from scipy.spatial import distance
from pyxdameraulevenshtein import damerau_levenshtein_distance

pd.set_option('display.max_rows', 500)

### Prepare Data

In [2]:
# read data, skip row that can't be clustered

df = pd.read_csv('automatica_task_environments_all.csv', header=0,
                 skip_blank_lines=True)

In [3]:
#df

### Define functions for prediction

In [3]:
def predict_sequence(objects, coordinates, start_coordinates, c, k, dimension=[3,]):
    ''' Predicts sequence based on required objects, object coordinates, start coordinates of subject,
        parameters (c+k) and dimensionality.
        Input: Objects, object coordinates, start coordinates, c, k, dimension
        Output: Sequence of objects as str
    '''
    prediction = []
    possible_items = dict.fromkeys(objects, 0) # generate dict from object list
    coord_index = 0
    start_coords = start_coordinates
    coords = coordinates
    new_coords = {}
    new_start_coords = []
    
    if dimension[0] == 3: # no changes if 3D
        new_coords = coords
        new_start_coords = start_coords
        
    elif dimension[0] == 2: # 2D: remove obsolete coordinate
        if dimension[1] == 'xy':
            new_coords = {key: value[:-1] for key, value in coords.items()}
            new_start_coords = [x[:-1] for x in start_coords]
                
        elif dimension[1] == 'xz':
            new_start_coords = [[x[0], x[-1]] for x in start_coords]
            
            for key, value in coords.items():
                new_value = (value[0], value[-1])
                new_coords[key] = new_value
        
        elif dimension[1] == 'yz':
            new_coords = {key: value[1:] for key, value in coords.items()}
            new_start_coords = [x[1:] for x in start_coords]
                
    elif dimension[0] == 1: # 1D: choose appropriate coordinate
        if dimension[1] == 'x':
            new_coords = {key: value[0] for key, value in coords.items()}
            new_start_coords = [x[0] for x in start_coords]
        
        elif dimension[1] == 'y':
            new_coords = {key: value[1] for key, value in coords.items()}
            new_start_coords = [x[1] for x in start_coords]
        
        elif dimension[1] == 'z':
            new_coords = {key: value[2] for key, value in coords.items()}
            new_start_coords = [x[2] for x in start_coords]
    
    while bool(possible_items) == True: # while dict not empty
        for obj in possible_items.keys():
            possible_items[obj] = ((distance.euclidean(
                                new_start_coords[coord_index], 
                                new_coords[obj])
                                ) ** k[obj]) * c[obj]
        #print(possible_items)                      
        minval = min(possible_items.values())
        minval = [k for k, v in possible_items.items() if v == minval]
        minval = random.choice(minval) # choose prediction randomly if multiple items have same cost
        prediction.append(minval)
        del possible_items[minval]
        coord_index += 1
        
    return prediction

In [4]:
def get_median(objects, coordinates, start_coordinates, c, k, dimension, sequence,n=10):
    ''' Returns average edit distance (Damerau-Levenshtein) for 100 trials of sequence prediction.
    '''
    edit_list = []

    for x in range(0,n):
        result = ''.join(predict_sequence(objects, coordinates, start_coordinates, c, k, dimension))
        dl = damerau_levenshtein_distance(sequence,result)
        edit_list.append(dl)
    
    #avg = np.mean(edit_list)
    median = np.median(edit_list)
    return median

In [6]:
def get_avg_editdist(data, dimensions=[[1,'x'],[1,'y'],[1,'z'],[2,'xy'],[2,'xz'],[2,'yz'],[3,'xyz']],n=10,
                    seq='sequence_original', coords='coordinates_original', error='error_original'):
    ''' Calculates average edit distance for all combinations of parameters (c, k, dimension).
        Input: Dataframe with objects, coordinates, start coordinates, object categories
        Output: Dataframe with edit distance results (col name: parameters used)
    '''
    results = pd.DataFrame()
        
    for row in range(0,len(data)):
        objects = list(data.at[row,seq])
        strong_k = list(data.at[row,'strong_k'].split(','))
        mid_k = list(data.at[row,'mid_k'].split(','))
        #food_k = list(data.at[row,'food_k'].split(','))
        food_k = []
        coordinates = {key: ast.literal_eval(value) for key, value in (elem.split(': ') for elem in data.at[row,coords].split(';'))}
        #print(coordinates)
        start_coordinates = list(ast.literal_eval(data.at[row,'start_coordinates']))
        sequence = str(data.at[row,seq])
        c1 = {obj: 1.2 for obj in objects}
        k1 = {obj: 1.0 for obj in objects}
        
        #for k2 in np.arange(1.1,2.0,0.1):
            #k_food = round(k2,2)
            #k1 = {obj: k_food if obj in food_k else 1.0 for obj in objects}
        
        for k in np.arange(0,0.9,0.1):
            k_strong = round(k,2)
            k_mid = round(k + 0.1,2)
            k1 = {obj: k_strong if obj in strong_k else k_mid if obj in mid_k else round(k1[obj],2) for obj in objects}
            
                #for c in np.arange(1.0,2.0,0.1):
                #    c = round(c, 1)
                #    c1 = {obj: 1.2 for obj in objects}
                
            for dim in dimensions:                
                # get average edit distance
                median = get_median(objects, coordinates, start_coordinates, c1, k1, dim, sequence, n)
                median = median / len(sequence)
                #params = 'c: ' + str(c) + '; k: ' + str(k_strong) + ',' + str(k_mid) + ',' + str(k_food) + '; ' + str(dim[1])
                params = 'c: 1.2' + '; k: ' + str(k_strong) + ',' + str(k_mid) + '; ' + str(dim[1])
                results.at[row,params] = median
                        
        results.at[row,'sequence'] = sequence
        results.at[row,'error'] = data.at[row,error]
        results.at[row,'ID'] = data.at[row,'ID']
                    
    return results

### Calculate edit distances, create df

In [7]:
# ~3 min for n=100
%time results_new = get_avg_editdist(df,n=100)

CPU times: user 3min 38s, sys: 572 ms, total: 3min 39s
Wall time: 3min 39s


In [8]:
results_new

,"c: 1.2; k: 0.0,0.1; x","c: 1.2; k: 0.0,0.1; y","c: 1.2; k: 0.0,0.1; z","c: 1.2; k: 0.0,0.1; xy","c: 1.2; k: 0.0,0.1; xz","c: 1.2; k: 0.0,0.1; yz","c: 1.2; k: 0.0,0.1; xyz","c: 1.2; k: 0.1,0.2; x","c: 1.2; k: 0.1,0.2; y","c: 1.2; k: 0.1,0.2; z",...,"c: 1.2; k: 0.8,0.9; x","c: 1.2; k: 0.8,0.9; y","c: 1.2; k: 0.8,0.9; z","c: 1.2; k: 0.8,0.9; xy","c: 1.2; k: 0.8,0.9; xz","c: 1.2; k: 0.8,0.9; yz","c: 1.2; k: 0.8,0.9; xyz",sequence,error,ID
0,0.666667,0.833333,0.833333,0.833333,0.833333,0.833333,0.833333,0.666667,0.833333,0.833333,...,0.666667,0.833333,0.833333,0.666667,0.833333,0.833333,0.666667,pocgkr,0.723,a1
1,0.818182,0.818182,0.818182,0.818182,0.818182,0.818182,0.818182,0.818182,0.818182,0.818182,...,0.727273,0.818182,0.636364,0.818182,0.727273,0.818182,0.818182,cgwpcffkkss,0.838,a3
2,0.615385,0.769231,0.692308,0.692308,0.615385,0.769231,0.692308,0.615385,0.769231,0.692308,...,0.615385,0.615385,0.692308,0.538462,0.615385,0.615385,0.538462,kfsfksppwwggg,0.862,a5
3,0.647059,0.647059,0.705882,0.647059,0.705882,0.647059,0.647059,0.647059,0.647059,0.705882,...,0.647059,0.647059,0.647059,0.647059,0.764706,0.647059,0.647059,ppfkswwkfsococggg,0.892,a11
4,0.888889,0.777778,0.888889,0.666667,0.888889,0.777778,0.666667,0.888889,0.666667,0.888889,...,0.777778,0.888889,0.666667,1.000000,0.777778,0.888889,1.000000,wptgkfsoc,0.806,a13
5,0.800000,0.800000,0.733333,0.800000,0.800000,0.800000,0.800000,0.800000,0.800000,0.733333,...,0.800000,0.866667,0.733333,0.800000,0.800000,0.866667,0.800000,pppwwwfffkkkggg,0.878,a16
6,0.833333,0.833333,0.833333,0.750000,0.833333,0.833333,0.750000,0.833333,0.833333,0.833333,...,0.750000,0.833333,0.833333,0.666667,0.750000,0.833333,0.666667,ppoowwcceerr,0.851,a19
7,0.818182,0.636364,0.727273,0.636364,0.818182,0.636364,0.636364,0.818182,0.636364,0.727273,...,0.818182,0.909091,0.636364,0.818182,0.818182,0.818182,0.818182,cooppreerkk,0.838,a22
8,0.866667,0.733333,0.733333,0.733333,0.866667,0.733333,0.733333,0.866667,0.733333,0.733333,...,0.800000,0.866667,0.666667,0.866667,0.733333,0.866667,0.866667,wwoppoffkkssskf,0.878,a26
9,0.666667,0.666667,0.666667,0.555556,0.666667,0.666667,0.555556,0.666667,0.666667,0.666667,...,0.666667,0.666667,0.666667,0.555556,0.666667,0.666667,0.555556,opwpwgcoc,0.806,a29


### Get lowest error, compare edit distances

In [9]:
def get_lowest_error(results):
    ''' Returns lowest error in dataframe and index of lowest error.
    '''
    for col in list(results):
        if col != 'sequence' and col != 'error' and col != 'ID':
            results.loc['mean',col] = results[col].mean()
            #results.loc['median',col] = results[col].median()
    lowest = min(results.loc['mean'])
    mean = list(results.loc['mean'])
    
    return lowest, results.columns[(results.loc['mean'] == lowest)], mean, results

In [10]:
lowest, lowest_idx, list_mean, results_mean = get_lowest_error(results_new)
lowest, lowest_idx

(0.7108442291242113,
 Index(['c: 1.2; k: 0.7,0.8; z', 'c: 1.2; k: 0.8,0.9; z'], dtype='object'))

In [16]:
#stdev for results better than baseline
np.std(results_new['c: 1.2; k: 0.2,0.3; xy'].loc[results_new['c: 1.2; k: 0.2,0.3; xy'] <= results_new['error']])

0.15251881149021448

In [26]:
# avg difference between best model and error
np.mean(results_new['error'][:-1] - results_new['c: 1.2; k: 0.2,0.3; xy'][:-1])

np.mean(results_new['error'][:-1].loc[results_new['sequence'] != 're'] - results_new['c: 1.2; k: 0.2,0.3; xy'][:-1].loc[results_new['sequence'] != 're'])

0.1109756006382961

In [13]:
np.nanmean(list_mean), np.nanmedian(list_mean)

(0.7257013595065066, 0.7286741771300594)

In [11]:
# Compare avg edit distance for xy and xyz
list_xy = []
list_xyz = []

for col in results_mean:
    if col != 'sequence' and col != 'error' and col != 'ID' and col.split(';')[2].strip() == 'xy':
        list_xy.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'error' and col != 'ID' and col.split(';')[2].strip() == 'xyz':
        list_xyz.append(results_mean.at['mean',col])
        
avg_xy = np.mean(list_xy)
med_xy = np.median(list_xy)
std_xy = np.std(list_xy)
print('Average xy: ' + str(avg_xy)  + ', stdev: ' + str(std_xy) + ', median: ' + str(med_xy))
        
avg_xyz = np.mean(list_xyz)
med_xyz = np.median(list_xyz)
std_xyz = np.std(list_xyz)
print('Average xyz: ' + str(avg_xyz) + ', stdev: ' + str(std_xyz) + ', median: ' + str(med_xyz))

Average xy: 0.7463640703956163, stdev: 0.02494377215716303, median: 0.7339491889815166
Average xyz: 0.7443640310033355, stdev: 0.02316864374903599, median: 0.7334706427432722


In [12]:
# Compare avg edit distances for x, y, z
list_x = []
list_y = []
list_z = []
list_xz = []
list_yz = []

for col in results_mean:
    if col != 'sequence' and col != 'error' and col != 'ID' and col.split(';')[2].strip() == 'x':
        list_x.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'error' and col != 'ID' and col.split(';')[2].strip() == 'y':
        list_y.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'error' and col != 'ID' and col.split(';')[2].strip() == 'z':
        list_z.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'error' and col != 'ID' and col.split(';')[2].strip() == 'xz':
        list_xz.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'error' and col != 'ID' and col.split(';')[2].strip() == 'yz':
        list_yz.append(results_mean.at['mean',col])

avg_xz = np.mean(list_xz)
avg_yz = np.mean(list_yz)
        
avg_x = np.mean(list_x)
lowest_x = min(list_x)
print('Average x: ' + str(avg_x) + ', min: ' + str(lowest_x)) 
        
avg_y = np.mean(list_y)
lowest_y = min(list_y)
print('Average y: ' + str(avg_y) + ', min: ' + str(lowest_y)) 
        
avg_z = np.mean(list_z)
lowest_z = min(list_z)
print('Average z: ' + str(avg_z) + ', min: ' + str(lowest_z)) 

print('Average xz: ' + str(avg_xz)) 
print('Average yz: ' + str(avg_yz)) 

Average x: 0.7724864675265073, min: 0.7505198088901183
Average y: 0.7649336285568537, min: 0.7333655221694015
Average z: 0.7383047218750183, min: 0.7108442291242113
Average xz: 0.7508103506604393
Average yz: 0.7685097418490971


### Statistical analysis

In [13]:
# Compare all error predictions for dimensions
stat, p = friedmanchisquare(list_x,list_y,list_z,list_xy,list_xz,list_yz,list_xyz)
print('Friedman: stat = %.3f, p = % 10.3E' % (stat, p))

Friedman: stat = 19.488, p =  3.414E-03


In [14]:
# 2D vs 3D median
stat, p = wilcoxon(list_xy, list_xyz, zero_method='wilcox')
print('Wilcoxon: W = %.3f, p = %.5f' % (stat, p))

Wilcoxon: W = 2.000, p = 0.13801


/home/zauri/files/anaconda3/lib/python3.7/site-packages/scipy/stats/morestats.py:2957: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/zauri/files/anaconda3/lib/python3.7/site-packages/scipy/stats/morestats.py:2971: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


### Plots for dataframe

In [15]:
# Define input for plots

c = [float(x[3:6]) for x in results_new.columns.tolist() if x != 'sequence' and x != 'error' and x != 'ID']
#k = [float(x[11:14]) for x in results_new.columns.tolist() if x != 'sequence']
k_strong = [float(x[11:14]) for x in results_new.columns.tolist() if x != 'sequence' and x != 'error' and x != 'ID']
k_mid = [float(x[15:18]) for x in results_new.columns.tolist() if x != 'sequence' and x != 'error' and x != 'ID']
dim = [x.strip() for x in results_new.columns[:-3].str.split(';').str[2]]
median = [x for x in results_mean.loc['mean'][:-1].tolist() if pd.notnull(x) and x != 'error' and x != 'ID']


In [21]:
%matplotlib qt

cm = ['red','blue','green','magenta','cyan','orange','grey']
dim_num = [0 if x=='x' else 1 if x=='y' else 2 if x=='z' else 3 if x=='xy' else 4 if x=='xz' else 5 if x=='yz' else 6 for x in dim]

cmap = matplotlib.colors.ListedColormap(cm)

ticks = ['x', 'y', 'z', 'xy', 'xz', 'yz', 'xyz']
norm = matplotlib.colors.BoundaryNorm(ticks, cmap.N)

# create figure, 3d grid, set background to white
fig2 = plt.figure(figsize=(12,8))
ax2 = fig2.add_subplot(111, projection='3d')
ax2.w_xaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_yaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_zaxis.set_pane_color((1.0,1.0,1.0,1.0))

# create plot w/ median edit distance
#img = ax2.scatter(c, k, median, alpha=0.5, s=38, c=dim_num, cmap=cmap)
img = ax2.scatter(k_strong, k_mid, median, alpha=0.5, s=38, c=dim_num, cmap=cmap)

# plot horizontal plane for baseline
#xx, yy = np.meshgrid(np.linspace(1,1.9), np.linspace(0,0.9))
xx, yy = np.meshgrid(np.linspace(0,0.9), np.linspace(0,0.9))
#yy = np.meshgrid(range(2), range(2))
zz = xx * 0 + np.mean(results_new['error'])
ax2.plot_surface(xx, yy, zz, alpha=0.5)

# set labels
ax2.set_ylabel('strong k', fontsize=20, labelpad=10)
ax2.set_xlabel('mid k', fontsize=20, labelpad=10)
ax2.set_zlabel('normalized edit distance', fontsize=20, labelpad=7)
#plt.title('Average edit distance (1 step)', fontsize=24)

# create colorbar
cb = plt.colorbar(img, cax = fig2.add_axes([0.9,0.3,0.03,0.4]))
cb.ax.set_yticklabels(ticks, fontsize=16)
plt.show()

/home/zauri/files/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: UserWarning: FixedFormatter should only be used together with FixedLocator


### Check for sequences worse than baseline

In [21]:
# get sequences from best model where performance worse than baseline
results_new['c: 1.2; k: 0.2,0.3; xy'].loc[results_new['c: 1.2; k: 0.2,0.3; xy'] >= 0.776]

print('Higher than avg. error: ' +
    str(len(results_new['c: 1.2; k: 0.2,0.3; xy'].loc[results_new['c: 1.2; k: 0.2,0.3; xy'] >= results_new['error']]))
    + ' of ' + \
    str(len(results_new['c: 1.2; k: 0.2,0.3; xy'])) + ' sequences')

Higher than avg. error: 14 of 81 sequences


In [25]:
worse_than_baseline = results_new['ID'].loc[results_new['c: 1.2; k: 0.2,0.3; xy'] >= results_new['error']]

worse_than_baseline

0       a1
1       a3
14     a40
24     a69
34    a100
36    a106
39    a114
41    a118
50    a156
51    a158
55    a174
57    a178
59    a185
68    a215
Name: ID, dtype: object

In [19]:
#%matplotlib inline

error_individual = df['error']
seqs = results_mean['sequence'][:-1].values
res = results_new['c: 1.2; k: 0.2,0.3; xy'][:-1].values
x = [x for x in range (0,len(seqs))]

plt.figure(figsize=(18,6))
plt.scatter(x, res, marker='o', label='model-generated sequences')
plt.scatter(x, error_individual, c='red', marker='o', linewidth=2, label='baseline for sequence length')
plt.plot(x, error_individual, c='red', alpha=0.5)
plt.plot((x,x),(error_individual,res), '--', c='black', alpha=0.6)


plt.title('same item type clustered, 0.662, c: 1.2; k: 0.2,0.3; xy], n=100', fontsize=16)
plt.xticks(x, seqs, rotation=90)
plt.ylabel('median edit distance', fontsize=14)
plt.xlabel('sequence', fontsize=14)
plt.legend()

#plt.savefig('plot_median_editdist_individualerrors_diff.png', bbox_inches='tight')
plt.show()

KeyError: 'error'

In [18]:
# Define input for 2D/3D plots

c2 = [float(x[3:6]) for x in results_new.columns.tolist() if x != 'sequence']
k2 = [float(x[11:14]) for x in results_new.columns.tolist() if x != 'sequence']
dim2 = [x.strip() for x in results_new.columns[:-1].str.split(';').str[2]]
median2 = [x for x in results_mean.loc['mean'].tolist() if pd.notnull(x)]

ValueError: could not convert string to float: 'or'

In [17]:
# Plot scatterplots for xy, xyz only

%matplotlib qt

cm2 = ['blue','green']
dim_num2 = [0 if x=='xy' else 1 for x in dim]

cmap2 = matplotlib.colors.ListedColormap(cm2)

ticks2 = ['xy', 'xyz']
norm = matplotlib.colors.BoundaryNorm(ticks2, cmap2.N)

# create figure, 3d grid, set background to white
fig3 = plt.figure(figsize=(12,8))
ax3 = fig3.add_subplot(111, projection='3d')
ax3.w_xaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax3.w_yaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax3.w_zaxis.set_pane_color((1.0,1.0,1.0,1.0))

# create plot w/ median edit distance
img2 = ax3.scatter(c2, k2, median2, alpha=0.5, s=38, c=dim_num2, cmap=cmap2)

# plot horizontal plane for baseline
xx, yy = np.meshgrid(np.linspace(1,1.9), np.linspace(0,0.9))
zz = xx * 0 + 0.603
ax3.plot_surface(xx, yy, zz, alpha=1.0)

# set labels
ax3.set_ylabel('parameter k', fontsize=20, labelpad=10)
ax3.set_xlabel('parameter c', fontsize=20, labelpad=10)
ax3.set_zlabel('normalized edit distance', fontsize=20, labelpad=7)

cb = plt.colorbar(img2, cax = fig3.add_axes([0.9,0.3,0.03,0.1]))
cb.ax.set_yticklabels(ticks2, fontsize=16)

plt.show()

NameError: name 'c2' is not defined